This exercise is to define Dummy classification baselines and use them as reference.

In [1]:
import pandas as pd

adult_census = pd.read_csv("../dataset/adult-census-numeric-all.csv")
data, target = adult_census.drop(columns="class"), adult_census["class"]